In [261]:
import pandas as pd
import openpyxl
import re

# Load the Excel file
file_path = 'Technical Test.xlsx'
xl = pd.ExcelFile(file_path)

# Load the 'Demographics' sheet
df_demo = xl.parse('Demographics')

## Task 1: Making the New NRIC ##
NRIC_list = df_demo["NRIC"]
digits = [] 
first_letter = []
last_letter = []

for NRIC_no in NRIC_list:
    num = re.findall(r'\d+',NRIC_no)
    digits.append(num[0])
    letters = re.findall(r'[A-Za-z]',NRIC_no)
    last_letter.append(str(letters[1]))

for x in digits:
    if x[0:2] == "20":
        first_letter.append("T")
    else:
        first_letter.append("S")

NRIC_new = [i+j+k for i,j,k in zip(first_letter,digits,last_letter)]

df_demo['New NRIC'] = NRIC_new


## Task 2: Coding the Gender ##
gender_list = df_demo["Gender"]
gender_code = []
for gender in gender_list:
    if gender == "F":
        gender_code.append(1)
    elif gender == "M":
        gender_code.append(2)
    else:
        gender_code.append("NA")

df_demo['Coding - Gender'] = gender_code


## Adding the Age Group ##
age_list = df_demo["Age"]
age_group = []
for age in age_list:
    if age <= 9:
        age_group.append("G1")
    elif age <= 19:
        age_group.append("G2")
    elif age <= 29:
        age_group.append("G3")
    elif age <= 39:
        age_group.append("G4")
    elif age <= 49:
        age_group.append("G5")
    elif age <= 59:
        age_group.append("G6")
    elif age <= 69:
        age_group.append("G7")
    elif age <= 79:
        age_group.append("G8")
    elif age <= 89:
        age_group.append("G9")
    else:
        age_group.append("G0")

df_demo["Age Group"] = age_group


## Task 4: Adding in Study Number ##
df_demo['Study Number'] = df_demo.groupby('Age Group').cumcount() + 1
df_demo['Study Number'] = df_demo['Age Group'] + '-' + df_demo['Study Number'].astype(str)


## Combining Demographics and Extra Information ##
df_extra = xl.parse('Extra information')
df_studydata = xl.parse('Study Data')

combined_demo_sd = pd.merge(df_studydata, df_demo[['Study Number', 'NRIC', 'New NRIC','Gender','Age','Marital Status']], left_on='Old NRIC', right_on='NRIC', how='left')
df_studydata['Study Number'] = combined_demo_sd['Study Number_y']
df_studydata['New NRIC'] = combined_demo_sd['New NRIC_y']
df_studydata['Gender'] = combined_demo_sd['Gender_y']
df_studydata['Age'] = combined_demo_sd['Age_y']
df_studydata['Marital Status'] = combined_demo_sd['Marital Status_y']

combined_extra_sd = pd.merge(df_studydata, df_extra[['NRIC','Address 1', 'Address 2', 'Contact Number','Ethnic Group']], left_on='Old NRIC', right_on='NRIC', how='left')
df_studydata['Address 1'] = combined_extra_sd['Address 1_y']
df_studydata['Address 2'] = combined_extra_sd['Address 2_y']
df_studydata['Contact Number'] = combined_extra_sd['Contact Number_y']
df_studydata['Ethnic Group'] = combined_extra_sd['Ethnic Group_y']

#print(df_studydata)

## Unique NRIC Numbers, Count of Unique NRIC, Missing Records Count## 

combined_demo_extra = pd.merge(df_demo, df_extra, on='NRIC', how='left')
#print(combined_demo_extra)
unique_nric = df_demo['New NRIC'].unique()
df_unique_nric = pd.DataFrame({
    'S/N': range(1, len(unique_nric) + 1),
    'NRIC': unique_nric})
#print(df_unique_nric)

unique_nric_count = len(unique_nric)
missing_nrics = list(set(df_demo['NRIC']) - set(df_extra['NRIC']))
#print(missing_nrics)

# Create a DataFrame from the list of IDs to lookup
lookup_df = pd.DataFrame({'NRIC': missing_nrics})
# Merge the DataFrames on 'ID' to get the names
result_df = pd.merge(lookup_df, df_demo, on='NRIC', how='left')
unique_result_df = result_df['New NRIC'].unique()
# Display the result
#print(result_df)
df_missing_records = pd.DataFrame({
    'S/N': range(1, len(result_df['New NRIC'].unique()) + 1),
    'NRIC': result_df['New NRIC'].unique()})
#print(df_missing_records)
missing_records_count = len(result_df['New NRIC'].unique())

## Pivot Table ##

# Generate counts for each category and sort them alphabetically
age_group_counts = df_demo['Age Group'].value_counts().reset_index().sort_values(by='index')
age_group_counts.columns = ['Age Group', 'Count']
age_group_counts.loc[len(age_group_counts)] = ['Total', age_group_counts['Count'].sum()]

marital_status_counts = df_demo['Marital Status'].value_counts().reset_index().sort_values(by='index')
marital_status_counts.columns = ['Marital Status', 'Count']
marital_status_counts.loc[len(marital_status_counts)] = ['Total', marital_status_counts['Count'].sum()]

gender_counts = df_demo['Gender'].value_counts().reset_index().sort_values(by='index')
gender_counts.columns = ['Gender', 'Count']
gender_counts.loc[len(gender_counts)] = ['Total', gender_counts['Count'].sum()]

df_pivot_table = pd.concat([age_group_counts, gender_counts, marital_status_counts])
#print(df_pivot_table)
df_pivot_table_arranged = df_pivot_table[['Age Group','Gender','Marital Status','Count']]
#print(df_pivot_table_arranged)


# Save the DataFrame back to Excel using openpyxl 
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    #Demographics Sheet
    df_demo.to_excel(writer, sheet_name='Demographics', index=False)
    #Study Data Sheet
    df_studydata.to_excel(writer, sheet_name='Study Data', index=False)
    #Pivot Table Sheet
    df_pivot_table_arranged.to_excel(writer, sheet_name='Pivot Table', index=False, startrow=0)
    
#Exception List Sheet
# Load the workbook again with openpyxl
wb = openpyxl.load_workbook(file_path)

# Select the sheet you want to work with
ws = wb['Exception List']

# Write data to specific cells
ws['B1'] = unique_nric_count
ws['E1'] = missing_records_count

rows = df_unique_nric.to_numpy().tolist()
columns = df_unique_nric.columns.tolist()

# Paste DataFrame into range starting at A4
start_row_A4 = 4
start_col_A4 = 1  # Column A

# Write column headers
for col_idx, column in enumerate(columns, start=start_col_A4):
    ws.cell(row=start_row_A4, column=col_idx, value=column)

# Write data
for r_idx, row in enumerate(rows, start=start_row_A4 + 1):
    for c_idx, value in enumerate(row, start=start_col_A4):
        ws.cell(row=r_idx, column=c_idx, value=value)

rows = df_missing_records.to_numpy().tolist()
columns = df_missing_records.columns.tolist()        
        
# Paste DataFrame into range starting at D4
start_row_D4 = 4
start_col_D4 = 4  # Column D

# Write column headers
for col_idx, column in enumerate(columns, start=start_col_D4):
    ws.cell(row=start_row_D4, column=col_idx, value=column)

# Write data
for r_idx, row in enumerate(rows, start=start_row_D4 + 1):
    for c_idx, value in enumerate(row, start=start_col_D4):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Save the workbook
wb.save(file_path)